In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms

from tqdm import tqdm
import random, time, os, sys

In [3]:
import sparse_nonlinear_lib as snl

In [4]:
device = torch.device("cuda:0")
# device = torch.device("cpu")

## For FMNIST dataset

In [5]:
train_transform = transforms.Compose([
            transforms.ToTensor(),
        ])
test_transform = transforms.Compose([
            transforms.ToTensor(),
        ])

train_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=True, download=True, transform=train_transform)
test_dataset = datasets.FashionMNIST(root="../../../../_Datasets/FMNIST/", train=False, download=True, transform=test_transform)

In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=50, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=50, shuffle=False, num_workers=2)

In [7]:
## demo of train loader
xx, yy = iter(train_loader).next()
xx.shape

torch.Size([50, 1, 28, 28])

# Model Comparision

## Pair Linear Mixing

In [8]:
def img2patch(x, input_dim=(1, 28, 28), patch_size=(7, 4)):
    y = nn.functional.unfold(x, 
                             kernel_size=patch_size, 
                             stride=patch_size
                            )
    return y

In [9]:
def patch2img(x, patch_size=(7, 4), input_dim=(1, 28, 28)):
    y = nn.functional.fold(x, (input_dim[-2], input_dim[-1]), 
                               kernel_size=patch_size, 
                               stride=patch_size
                              )
    return y

In [10]:
img2patch(torch.randn(2, 1, 28, 28)).shape

torch.Size([2, 28, 28])

In [11]:
patch2img(torch.randn(2, 7* 4, 7*4)).shape

torch.Size([2, 1, 28, 28])

In [12]:
(7*4)**2, (8*4)**2

(784, 1024)

1. Linearize by expanding the dimension of folded image.

## Final Model

In [13]:
class FMNIST_BlockMLP(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), patch_size=(7, 7)):
        super().__init__()
        self.patch_size = patch_size
        self.patch_linear = nn.Linear(img_size[0]*patch_size[0]*patch_size[1], 8*8)
#         self.dim_sel = snl.DimensionSelector(8*8*4*4, 8*4*8*4)

#         self.block_mlp = snl.BlockMLP_MixerBlock(8*4*8*4, 8*4, hidden_layers_ratio=[4])
        self.block_mlp = snl.BlockMLP_MixerBlock(8*4*8*4, 2, hidden_layers_ratio=[10])
        self.bn = nn.BatchNorm1d(8*4*8*4)
        self.fc = nn.Linear(8*4*8*4, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = img2patch(x, input_dim=(1, 28, 28), patch_size=self.patch_size)
#         print(x.shape)
        x = self.patch_linear(x.transpose(-2, -1)).reshape(bs, -1)
#         print(x.shape)
#         x = self.dim_sel(x)
#         print(x.shape)
        x = self.block_mlp(x)
#         print(x.shape)
        x = self.bn(x)
        x = self.fc(x)
        return x

In [14]:
class FMNIST_OrdMLP(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28)):
        super().__init__()
        self.input_dim = np.prod(img_size)
        M = 1500#354 #1024
        self.l0 = nn.Linear(self.input_dim, M)
        self.l1 = nn.Linear(M, 10)
        self.elu = nn.ELU()
        self.ln = nn.LayerNorm(M)
        
    def forward(self, x):
        bs = x.shape[0]
        x = self.l0(x.reshape(bs, -1))
        x = self.ln(x)
        x = self.elu(x)
        x = self.l1(x)
        return x

In [15]:
class FMNIST_PairBilinear(nn.Module):
    
    def __init__(self, img_size=(1, 28, 28), patch_size=(7, 7), grid_width=5):
        super().__init__()
        self.patch_size = patch_size
        self.patch_linear = nn.Linear(img_size[0]*patch_size[0]*patch_size[1], 8*8)
#         self.dim_sel = snl.DimensionSelector(8*8*4*4, 8*4*8*4)
#         self.ln = nn.LayerNorm(8*4*8*4)
        self.bn = nn.BatchNorm1d(8*4*8*4)

        self.block_func = snl.PairBilinear_MixerBlock(8*4*8*4, 8*4*8*4, grid_width=grid_width)#.double()
        self.fc = nn.Linear(8*4*8*4, 10)
        
    def forward(self, x):
        bs = x.shape[0]
        x = img2patch(x, input_dim=(1, 28, 28), patch_size=self.patch_size)
        x = self.patch_linear(x.transpose(-2, -1)).reshape(bs, -1)
#         x = self.block_func(x.type(torch.double)).to(torch.float32)
        x = self.block_func(x)
        x = self.bn(x)
#         x = self.ln(x)
        x = self.fc(x)
        return x

In [16]:
block_func = snl.PairBilinear_MixerBlock(8*4*8*4, 8*4*8*4, grid_width=10).type(torch.double).to(device)
# block_func.pairwise_mixing[0].pairW.dtype
for p in block_func.parameters():
    print(p.shape, p.dtype, p.device)

torch.Size([1024]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0
torch.Size([512, 2, 2]) torch.float64 cuda:0
torch.Size([512, 2, 10, 10]) torch.float64 cuda:0


In [17]:
block_func(torch.randn(2, 1024).to(device).double())

tensor([[ 0.6331,  0.0728,  0.9186,  ..., -0.0287, -0.1448,  0.8178],
        [-1.3230,  0.3096, -0.4694,  ..., -0.3249,  1.9116,  0.0740]],
       device='cuda:0', dtype=torch.float64, grad_fn=<ViewBackward0>)

In [18]:
# a = torch.randn_like(block_func.pairwise_mixing[0].Y)
try:
    a = model.block_func.pairwise_mixing[0].Y
except:
    a = block_func.pairwise_mixing[0].Y

In [19]:
a

Parameter containing:
tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ...,

In [20]:
init0 = a[:,:,:,:1]
init1 = a[:,:,:1,:]
init0.shape, init1.shape

(torch.Size([512, 2, 10, 1]), torch.Size([512, 2, 1, 10]))

In [21]:
a0 = torch.diff(a, dim=-1)
a0.shape

torch.Size([512, 2, 10, 9])

In [22]:
a0[0]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0',
       dtype=torch.float64, grad_fn=<Se

In [23]:
a0.data.max()

tensor(0., device='cuda:0', dtype=torch.float64)

In [24]:
a1 = torch.diff(a, dim=-2)
a1.shape

torch.Size([512, 2, 9, 10])

In [25]:
a1[0]

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], device='cuda:0',
       dtype=torch.float64, grad_fn=<SelectBackward0>)

In [26]:
a1.data.max()

tensor(0., device='cuda:0', dtype=torch.float64)

In [27]:
# model.block_func.pairwise_mixing[0].dy

In [28]:
_a0 = torch.cat([init0, a0], dim=-1)
_a0.shape

torch.Size([512, 2, 10, 10])

In [29]:
_a1 = torch.cat([init1, a1], dim=-2)
_a1.shape

torch.Size([512, 2, 10, 10])

In [30]:
f0 = torch.cumsum(_a0, dim=-1)

In [31]:
f1 = torch.cumsum(_a1, dim=-2)

In [32]:
f = (f0+f1)/2

In [33]:
torch.abs(f-a).mean()

tensor(0., device='cuda:0', dtype=torch.float64, grad_fn=<MeanBackward0>)

In [34]:
x = torch.randn(2, 1024).double().to(device)
block_func(x)
# x.dtype

tensor([[ 0.7227,  1.7482,  2.2501,  ...,  0.7609,  0.5850, -0.5791],
        [-0.0686,  1.0620,  1.4424,  ..., -0.2611,  0.4803,  1.3782]],
       device='cuda:0', dtype=torch.float64, grad_fn=<ViewBackward0>)

In [35]:
# model = FMNIST_BlockMLP(img_size=(1, 28, 28), patch_size=(7, 7))
# model = FMNIST_OrdMLP(img_size=(1, 28, 28))
model = FMNIST_PairBilinear(img_size=(1, 28, 28), patch_size=(7, 7), grid_width=7)

model = model.to(device)

In [36]:
model

FMNIST_PairBilinear(
  (patch_linear): Linear(in_features=49, out_features=64, bias=True)
  (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block_func): PairBilinear_MixerBlock(
    (selector): BiasLayer: [1024]
    (pairwise_mixing): ModuleList(
      (0): PairLinear: [1024 -> 1024] (grid: 7)
      (1): PairLinear: [1024 -> 1024] (grid: 7)
      (2): PairLinear: [1024 -> 1024] (grid: 7)
      (3): PairLinear: [1024 -> 1024] (grid: 7)
      (4): PairLinear: [1024 -> 1024] (grid: 7)
      (5): PairLinear: [1024 -> 1024] (grid: 7)
      (6): PairLinear: [1024 -> 1024] (grid: 7)
      (7): PairLinear: [1024 -> 1024] (grid: 7)
      (8): PairLinear: [1024 -> 1024] (grid: 7)
      (9): PairLinear: [1024 -> 1024] (grid: 7)
    )
    (reducer): Identity()
  )
  (fc): Linear(in_features=1024, out_features=10, bias=True)
)

In [37]:
model(torch.randn(2, 1, 28, 28).to(device)).shape

torch.Size([2, 10])

In [38]:
print("number of params: ", sum(p.numel() for p in model.parameters()))

number of params:  538762


In [39]:
## BLOCK MLP (1024/64) number of params:  281738
## ORDINARY MLP (784, 1024) number of params:  814090
## PAIR BILINEAR has 1058954

In [40]:
784*784

614656

## Training

In [41]:
 ## debugging to find the good classifier/output distribution.
# model_name = 'block_mlp_mixer_fmnist_v0'
# model_name = 'ord_mlp_mixer_fmnist_v0'
model_name = 'pair_bilinear_mixer_fmnist_v0'

In [42]:
EPOCHS = 50
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

optimizer = torch.optim.Adam(model.parameters(), lr=0.00003)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [43]:
## Following is copied from 
### https://github.com/kuangliu/pytorch-cifar/blob/master/main.py

# Training
def train(epoch):
    model.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(tqdm(train_loader)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    print(f"[Train] {epoch} Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    return

In [44]:
best_acc = -1
def test(epoch):
    global best_acc
    model.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(tqdm(test_loader)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
    print(f"[Test] {epoch} Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f} {correct}/{total}")
    
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'model': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f'./models/{model_name}.pth')
        best_acc = acc

In [45]:
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
resume = False

if resume:
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('./models'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load(f'./models/{model_name}.pth')
    model.load_state_dict(checkpoint['model'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

In [46]:
### Train the whole damn thing

for epoch in range(start_epoch, start_epoch+EPOCHS): ## for 200 epochs
    train(epoch)
    test(epoch)
    scheduler.step()

100%|████████████████████████████████████████████████| 1200/1200 [00:08<00:00, 147.05it/s]


[Train] 0 Loss: 0.783 | Acc: 74.672 44803/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 273.77it/s]


[Test] 0 Loss: 0.515 | Acc: 81.930 8193/10000
Saving..


100%|████████████████████████████████████████████████| 1200/1200 [00:08<00:00, 144.78it/s]


...

[Test] 48 Loss: 0.323 | Acc: 89.130 8913/10000


100%|████████████████████████████████████████████████| 1200/1200 [00:08<00:00, 148.06it/s]


[Train] 49 Loss: 0.096 | Acc: 97.328 58397/60000


100%|██████████████████████████████████████████████████| 200/200 [00:00<00:00, 270.95it/s]

[Test] 49 Loss: 0.328 | Acc: 89.070 8907/10000


In [47]:
best_acc

89.26

In [48]:
checkpoint = torch.load(f'./models/{model_name}.pth')
best_acc = checkpoint['acc']
start_epoch = checkpoint['epoch']

best_acc, start_epoch

(89.26, 39)

In [49]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [50]:
model

FMNIST_PairBilinear(
  (patch_linear): Linear(in_features=49, out_features=64, bias=True)
  (bn): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (block_func): PairBilinear_MixerBlock(
    (selector): BiasLayer: [1024]
    (pairwise_mixing): ModuleList(
      (0): PairLinear: [1024 -> 1024] (grid: 7)
      (1): PairLinear: [1024 -> 1024] (grid: 7)
      (2): PairLinear: [1024 -> 1024] (grid: 7)
      (3): PairLinear: [1024 -> 1024] (grid: 7)
      (4): PairLinear: [1024 -> 1024] (grid: 7)
      (5): PairLinear: [1024 -> 1024] (grid: 7)
      (6): PairLinear: [1024 -> 1024] (grid: 7)
      (7): PairLinear: [1024 -> 1024] (grid: 7)
      (8): PairLinear: [1024 -> 1024] (grid: 7)
      (9): PairLinear: [1024 -> 1024] (grid: 7)
    )
    (reducer): Identity()
  )
  (fc): Linear(in_features=1024, out_features=10, bias=True)
)